In [48]:
import pandas as pd
from collections import Counter
from scripts.load_data import extract_labeled_tokens, label_mapping, read_tsv_file

In [49]:
# paths to the data
path_train_original = "data/da_news/da_news_train.tsv"
path_dev_original = "data/da_news/da_news_dev.tsv"
path_test_original = "data/da_news/da_news_test.tsv"

path_train_emerging = "data/no_overlap_da_news/da_news_train.tsv"
path_dev_emerging = "data/no_overlap_da_news/da_news_dev.tsv"
path_test_emerging = "data/no_overlap_da_news/da_news_test.tsv"

In [50]:
# create mapping
label2id, id2label = label_mapping(path_train_original)

In [51]:
# read in the datasets
train_data_original = read_tsv_file(path_train_original, label2id)
dev_data_original = read_tsv_file(path_dev_original, label2id)
test_data_original = read_tsv_file(path_test_original, label2id)

train_data_emerging = read_tsv_file(path_train_emerging, label2id)
dev_data_emerging = read_tsv_file(path_dev_emerging, label2id)
test_data_emerging = read_tsv_file(path_test_emerging, label2id)

# Sizes of the datasets before and after emerging enitities

In [52]:
print("Before emerging entities")
print('\tSize of training set:', len(train_data_original))
print('\tSize of dev set:', len(dev_data_original))
print('\tSize of test set:', len(test_data_original))

print("\nAfter emerging entities")
print('\tSize of training set:', len(train_data_emerging))
print('\tSize of dev set:', len(dev_data_emerging))
print('\tSize of test set:', len(test_data_emerging))

Before emerging entities
	Size of training set: 4383
	Size of dev set: 564
	Size of test set: 565

After emerging entities
	Size of training set: 4411
	Size of dev set: 549
	Size of test set: 552


# Token Overlap of Original Data

In [53]:
# token overlap
def check_token_overlap(train_data, dev_data, test_data):
    train_tokens = extract_labeled_tokens(train_data)
    dev_tokens = extract_labeled_tokens(dev_data)
    test_tokens = extract_labeled_tokens(test_data)

    print('overlap between train and dev:', len(train_tokens & dev_tokens))
    print('overlap between dev and test:', len(dev_tokens & test_tokens))
    print('overlap between train and test:', len(train_tokens & test_tokens))

check_token_overlap(train_data_original, dev_data_original, test_data_original)

overlap between train and dev: 256
overlap between dev and test: 78
overlap between train and test: 219


# Checking for -part and -deriv in Original Data

In [54]:
# loading data sets with Pandas
news_train = pd.read_csv("data/da_news/da_news_train.tsv", sep="\t", header=None, names=["Token", "Tag", "Nested"], usecols=["Token", "Tag"])
news_test = pd.read_csv("data/da_news/da_news_test.tsv", sep="\t", header=None, names=["Token", "Tag", "Nested"], usecols=["Token", "Tag"])
news_dev = pd.read_csv("data/da_news/da_news_dev.tsv", sep="\t", header=None, names=["Token", "Tag", "Nested"], usecols=["Token", "Tag"], engine="python", on_bad_lines="skip")

In [55]:
# frequency of part/deriv tags in training data
part_deriv = news_train["Tag"].str.contains("part|deriv").sum()

non_O_tag = (news_train["Tag"] != "O").sum()

print(part_deriv/non_O_tag*100)

12.109955423476968


In [56]:
# number of part/deriv by type in training data
tags = ["B-LOCpart", "B-LOCderiv", "B-ORGpart", "B-ORGderiv", "B-PERpart", "B-PERderiv", "B-MISCpart", "B-MISCderiv",
        "I-LOCpart", "I-LOCderiv", "I-ORGpart", "I-ORGderiv", "I-PERpart", "I-PERderiv", "I-MISCpart", "I-MISCderiv"]

for tag in tags:
    count = (news_train["Tag"]==tag).sum()
    print(f"{tag}: {count}")

B-LOCpart: 17
B-LOCderiv: 232
B-ORGpart: 51
B-ORGderiv: 4
B-PERpart: 6
B-PERderiv: 1
B-MISCpart: 13
B-MISCderiv: 2
I-LOCpart: 0
I-LOCderiv: 0
I-ORGpart: 0
I-ORGderiv: 0
I-PERpart: 0
I-PERderiv: 0
I-MISCpart: 0
I-MISCderiv: 0


# Counting length of LOC and ORG entities in Original Data

### Counting LOC

In [57]:
# counting the token length of locations
def count_loc(tags):
    i = 0
    LOC_length = []

    while i < len(tags):
        if tags[i] == 'B-LOC':
            length = 1  
            i += 1
            while i < len(tags) and tags[i] == 'I-LOC':
                length += 1
                i += 1
            LOC_length.append(length)
        else:
            i += 1

    return Counter(LOC_length)

In [58]:
# length of tokens in train
count_loc(news_train["Tag"].tolist())

Counter({1: 388, 2: 27, 3: 3, 4: 2})

In [59]:
# length of tokens in dev
count_loc(news_dev["Tag"].tolist())

Counter({1: 52, 2: 4})

In [60]:
# length of tokens in test
count_loc(news_test["Tag"].tolist())

Counter({1: 47})

### Counting ORG

In [61]:
# counting the token length of organizations
def count_org(tags):
    i = 0
    ORG_length = []

    while i < len(tags):
        if tags[i] == 'B-ORG':
            length = 1  
            i += 1
            while i < len(tags) and tags[i] == 'I-ORG':
                length += 1
                i += 1
            ORG_length.append(length)
        else:
            i += 1

    return Counter(ORG_length)

In [62]:
# length of tokens in train
count_org(news_train["Tag"].tolist())

Counter({1: 260, 2: 102, 3: 48, 4: 6, 5: 1})

In [63]:
# length of tokens in dev
count_org(news_dev["Tag"].tolist())

Counter({1: 36, 2: 7, 3: 4})

In [64]:
# length of tokens in test
count_org(news_test["Tag"].tolist())

Counter({1: 35, 2: 21, 3: 5})

# Named Entity Distribution in Original Data

In [65]:
# Counting the occurence of the NER tags in each set
NER_tags = ["B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC", "O"]

for tag in NER_tags:
    count = (news_train["Tag"] == tag).sum()
    print(f"{tag}: {count}")

for tag in NER_tags:
    count = (news_dev["Tag"] == tag).sum()
    print(f"{tag}: {count}")

for tag in NER_tags:
    count = (news_test["Tag"] == tag).sum()
    print(f"{tag}: {count}")

B-PER: 596
I-PER: 414
B-ORG: 417
I-ORG: 220
B-LOC: 420
I-LOC: 41
B-MISC: 138
I-MISC: 120
O: 38146
B-PER: 88
I-PER: 58
B-ORG: 47
I-ORG: 15
B-LOC: 56
I-LOC: 4
B-MISC: 17
I-MISC: 20
O: 5196
B-PER: 63
I-PER: 54
B-ORG: 61
I-ORG: 34
B-LOC: 47
I-LOC: 0
B-MISC: 19
I-MISC: 13
O: 4544


### Counting O

# Named Entity Distribution in Non-Overlapping Data

In [66]:
# Counting the occurence of the NER tags in each set
def count_tag(data, tag_str):
    return sum(tag == tag_str for sentence in data for tag in sentence["ner_tags"])

NER_tags = ["B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC", "O"]

for tag in NER_tags:
    count = count_tag(train_data, tag)
    print(f"{tag}: {count}")

for tag in NER_tags:
    count = count_tag(dev_data, tag)
    print(f"{tag}: {count}")

for tag in NER_tags:
    count = count_tag(test_data, tag)
    print(f"{tag}: {count}")

B-PER: 1224
I-PER: 878
B-ORG: 809
I-ORG: 461
B-LOC: 839
I-LOC: 76
B-MISC: 282
I-MISC: 251
O: 75558
B-PER: 162
I-PER: 105
B-ORG: 93
I-ORG: 40
B-LOC: 102
I-LOC: 12
B-MISC: 30
I-MISC: 29
O: 9759
B-PER: 178
I-PER: 139
B-ORG: 156
I-ORG: 55
B-LOC: 85
I-LOC: 4
B-MISC: 43
I-MISC: 35
O: 9328


# Sizes of the datasets before and after emerging enitities

In [67]:
print("Before emerging entities")
print('\tSize of training set:', len(train_data_original))
print('\tSize of dev set:', len(dev_data_original))
print('\tSize of test set:', len(test_data_original))

print("\nAfter emerging entities")
print('\tSize of training set:', len(train_data_emerging))
print('\tSize of dev set:', len(dev_data_emerging))
print('\tSize of test set:', len(test_data_emerging))

Before emerging entities
	Size of training set: 4383
	Size of dev set: 564
	Size of test set: 565

After emerging entities
	Size of training set: 4411
	Size of dev set: 549
	Size of test set: 552
